In [ ]:
!ollama pull mxbai-embed-large

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaEmbeddings

# Configure embeddings
ollama_embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Initialize FAISS vector store with text data
vectorstore = FAISS.from_texts(
    [
        "Cats are geniuses at claiming boxes as their own.",
        "Dogs have successfully trained humans to take them for walks.",
        "Cats aren't fond of water, but the water in a human's cup is an exception.",
        "Dogs follow cats around, eager to befriend them.",
        "Cats consider laser pointers their arch-nemesis.",
    ],
    embedding=ollama_embeddings,
)
# Convert vector store to retriever
retriever = vectorstore.as_retriever()

# Define prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# Initialize chat prompt from template
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
!ollama pull llama3.2:1b

In [ ]:
from langchain_ollama import ChatOllama

# Initialize Ollama chat model

base_url = "http://localhost:11434"
model = 'llama3.2:1b'
ollama_model = ChatOllama(
    base_url=base_url,
    model=model,
    temperature=0.5,
    num_predict=512
    #timeout=None,
    # max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
   
)


# Format retrieved documents
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# Build retrieval chain
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_model  # Use Ollama model for inference
    | StrOutputParser()
)

In [ ]:
# Query retrieval chain
retrieval_chain.invoke("What kind of objects do cats like?")

In [ ]:
# Query retrieval chain
retrieval_chain.invoke("What do dogs like?")

In [ ]:
# without context

retrieval_chain = prompt | ollama_model  | StrOutputParser()

In [ ]:
retrieval_chain.invoke({"question":"What do dogs like?", "context":""})